In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/yolov5
!find "/content/drive/MyDrive/healthcare_data/labels/train" -size 0 | wc -l

/content/drive/MyDrive/yolov5


In [ ]:
!rm /content/drive/MyDrive/healthcare_data_slice/images/train/*
!rm /content/drive/MyDrive/healthcare_data_slice/images/val/*
!rm /content/drive/MyDrive/healthcare_data_slice/labels/train/*
!rm /content/drive/MyDrive/healthcare_data_slice/labels/val/*

In [6]:
!find /content/drive/MyDrive/healthcare_data_slice/images/train | wc -l
!find /content/drive/MyDrive/healthcare_data_slice/images/val  | wc -l
!find /content/drive/MyDrive/healthcare_data_slice/labels/train | wc -l
!find /content/drive/MyDrive/healthcare_data_slice/labels/val | wc -l

158
44
187
47


In [5]:
import json
import pandas as pd 
import os
import numpy as np 
import shutil
import cv2
import random
slice_size=750
#SOMETHING WRONG S REGIONS DONT LOOK RIGHT
def vgg_2_yolo(region, img_height, img_width):
	if region["name"] =="rect":
		x = ((region["x"]%slice_size)+(region["width"]/2))/slice_size
		y = ((region["y"]%slice_size)+(region["height"]/2))/slice_size
		width = region["width"]/slice_size
		height = region["height"]/slice_size
		return min(x,1), min(1,y), min(1,width), min(1,height), int((region["x"]+region["width"]/2)//slice_size), int((region["y"]+region["height"]/2)//slice_size)

	if region["name"]=="circle":
		x = (region["cx"]%slice_size)/ slice_size
		y =  (region["cy"] %slice_size)/ slice_size
		width = region["r"]*2 / slice_size
		height = region["r"]*2 / slice_size
		return min(1,x), min(1,y) , min(1, width), min(1,height), int(region["cx"]//slice_size), int(region["cy"]//slice_size)
	if region["name"]=="ellipse":
		x = (region["cx"]%slice_size)/ slice_size
		y =  (region["cy"]%slice_size) / slice_size
		width = region["rx"]*2 / slice_size
		height = region["ry"]*2 / slice_size
		return min(1,x), min(y,1) , min(1,width), min(1,height), int(region["cx"]//slice_size), int(region["cy"]//slice_size)
visited=[]
counter=0

for folders in os.listdir("/content/drive/MyDrive/AI_FOR_HEALTHCARE_DATASETS"):
	if folders not in ["Batch 07", "Batch 06", "Batch 08"]:
		continue
	for files in os.listdir("/content/drive/MyDrive/AI_FOR_HEALTHCARE_DATASETS/"+folders):
		
		if files.strip()[-1]!='v':
			continue
		csv= pd.read_csv("/content/drive/MyDrive/AI_FOR_HEALTHCARE_DATASETS/"+folders+"/"+files)
		array=["train","train","train","train","train","train","train","train","val", "val"]
		try:
			for index, row in csv.iterrows():
				
				img = cv2.imread("/content/drive/MyDrive/AI_FOR_HEALTHCARE_DATASETS/"+folders+"/"+row["filename"])
				img_height, img_width ,c= img.shape
				index = random.randint(0,9)

				region = json.loads(row["region_shape_attributes"])
				if region =={}:
					#a= random.randint(0,10)
					#if a> 8:
					continue
					#	f = open("/content/drive/MyDrive/healthcare_data_slice/labels/"+array[index]+"/"+row["filename"].strip().split('.')[0]+"b"+str(b)+"a"+str(a)+".txt", 'a')
					#	f.close()
					
				else:
					x, y, width,height, a, b = vgg_2_yolo(region, img_height, img_width)
					f = open("/content/drive/MyDrive/healthcare_data_slice/labels/"+array[index]+"/"+row["filename"].strip().split('.')[0]+"b"+str(b)+"a"+str(a)+".txt", 'a')
					f.write("0 "+str(x)+" "+ str(y)+" "+str(width)+" "+str(height)+"\n")
				if True:
						print(counter)
						nh = img_height//slice_size  
						nw = img_width//slice_size  
						hinterval = 1/nh
						winterval = 1/nw
						newimg = [[img[y*slice_size:(y+1)*slice_size , x*slice_size:(x+1)*slice_size] for y in range(0, nh)] for x in range(0, nw) ]
	#newimg = [[img[y*512:(y+1)*512 , x*512:(x+1)*512] if y!=nh else img[y*512:h , x*512:(x+1)*512] for y in range(0, nh+1)] if x != nw else [img[y*512:(y+1)*512 , x*512:w] for y in range(0, nh+1)] for x in range(0, nw+1) ]


						for i in range(0, nh):
							for j in range(0, nw):
								try:
									if row["filename"].strip().split('.')[0]+"b"+str(j)+"a"+str(i)+".txt" in os.listdir("/content/drive/MyDrive/healthcare_data_slice/labels/"+array[index]):
										
										cv2.imwrite("/content/drive/MyDrive/healthcare_data_slice/images/"+array[index]+"/"+row["filename"].strip().split('.')[0]+"b"+str(j)+"a"+str(i)+".png" , newimg[i][j])
									
								except Exception as d:
									print(row["filename"] )
									print("d")
						#shutil.copy("/content/drive/MyDrive/AI_FOR_HEALTHCARE_DATASETS/"+folders+"/"+row["filename"], "/content/drive/MyDrive/healthcare_data_slice/images/"+array[index]+"/")
						visited.append(row["filename"])
				counter+=1
		except Exception as e:
			print(e)




	

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
10001625730_523829474.png
d
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102


In [7]:
#for file in os.listdir("/content/drive/MyDrive/healthcare_data/labels/train"):
 #   if file.strip().split(".")[0]+".PNG" not in os.listdir("/content/drive/MyDrive/healthcare_data/images/train"):
  #      print(file)
        #os.remove("/content/drive/MyDrive/healthcare_data/labels/train")
train_list_images =os.listdir("/content/drive/MyDrive/healthcare_data_slice/images/train")
train_list_labels =os.listdir("/content/drive/MyDrive/healthcare_data_slice/labels/train")
train_list_images = [x.strip().split(".")[0]+".txt" for x in train_list_images]

for i in (list(set(train_list_labels)-set(train_list_images) )):
    print(i)
    os.remove("/content/drive/MyDrive/healthcare_data_slice/labels/train/"+i)
val_list_images =os.listdir("/content/drive/MyDrive/healthcare_data_slice/images/val")
val_list_labels =os.listdir("/content/drive/MyDrive/healthcare_data_slice/labels/val")
val_list_images = [x.strip().split(".")[0]+".txt" for x in val_list_images]
for i in (list(set(val_list_labels)-set(val_list_images) )):
    print(i)
    os.remove("/content/drive/MyDrive/healthcare_data_slice/labels/val/"+i)
print(len(os.listdir("/content/drive/MyDrive/healthcare_data_slice/images/train")))
print(len(os.listdir("/content/drive/MyDrive/healthcare_data_slice/images/val")))
print(len(os.listdir("/content/drive/MyDrive/healthcare_data_slice/labels/train")))
print(len(os.listdir("/content/drive/MyDrive/healthcare_data_slice/labels/val")))

10001608491_531514311b2a3.txt
10001677850_525861313b3a4.txt
10001597059_521717081b1a3.txt
10001595358_521608534b2a4.txt
10001591359_521257476b2a4.txt
10001692907_526823255b2a4.txt
10001683203_526099064b3a4.txt
10001591359_521257476b2a3.txt
10001695249_526989602b2a4.txt
10001601929_522031052b4a1.txt
10001620263_523360185b1a3.txt
10001591036_521199996b3a4.txt
10001646342_525288063b1a3.txt
10001623985_523595019b3a4.txt
10001562155_519434411b2a4.txt
10001625283_523741066b2a4.txt
10001557186_519129264b2a3.txt
10001696378_527053497b2a4.txt
10001648962_525439637b2a4.txt
10001578293_520441272b2a4.txt
10001650184_525476723b2a4.txt
10001625761_523836442b2a4.txt
10001646326_525275374b2a4.txt
10001640975_524823067b2a4.txt
10001569186_519890798b2a4.txt
10001625730_523829474b3a4.txt
10001642991_525082493b2a4.txt
10001646359_525288430b2a4.txt
10001619511_523310524b2a3.txt
10001651933_525598007b2a4.txt
10001564221_519597885b3a4.txt
10001623985_523595018b3a4.txt
157
43
157
43
